In [2]:
import pandas as pd
df = pd.read_csv("/home/hp/Desktop/ML-using-Python/tennis.csv")
df

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [3]:
import math
from treelib import Node, Tree

def igain(df, cname):
    dict = df.groupby([cname]).groups
    val = 0
    for key in dict.keys():
        dfx = df.loc[dict[key].values.tolist()]
        tval = 0
        c1 = len(dfx[dfx.play == "yes"])
        c2 = len(dfx[dfx.play == "no"])
        if c1 != 0:
            pb = float(c1)/(c1+c2)
            tval += pb * math.log(pb)
        if c2 != 0:
            pb = float(c2)/(c1+c2)
            tval += pb * math.log(pb)    
            
        val += tval*(float(len(dfx)) / len(df))
    return val

In [52]:
def decision_tree(D, tree, parent, value, c):
    mgain = -10000
    mcol = "111"
    
    if len(D.columns.values) == 1:
        u = len(D[D.play == 'yes'])
        v = len(D) - u        
        if u >= v:
            print c, "yes"
        else:
            print c, "no"
        return 111    
        
    for col in range(len(D.columns.values)-1): #assume atleast 2 columns
        cname = D.columns.values[col]
        gain = igain(D, cname)
        if gain > mgain:
            mgain = gain
            mcol = cname                    
    
    dict = D.groupby([mcol]).groups
    for key in dict.keys():
        dfx = D.loc[dict[key].values.tolist()]
        dfx = dfx.drop([mcol], axis = 1)
        print c, str(mcol) + " == " + str(key)
        decision_tree(dfx, tree, mcol, key, c-1)

In [53]:
tree = Tree()
decision_tree(df, tree, "root", "root", 5)
#http://www.saedsayad.com/decision_tree.htm

5 outlook == rainy
4 windy == False
3 temp == mild
2 humidity == high
1 yes
2 humidity == normal
1 yes
3 temp == cool
2 humidity == normal
1 yes
4 windy == True
3 temp == mild
2 humidity == high
1 no
3 temp == cool
2 humidity == normal
1 no
5 outlook == overcast
4 temp == hot
3 humidity == high
2 windy == False
1 yes
3 humidity == normal
2 windy == False
1 yes
4 temp == mild
3 humidity == high
2 windy == True
1 yes
4 temp == cool
3 humidity == normal
2 windy == True
1 yes
5 outlook == sunny
4 humidity == high
3 temp == hot
2 windy == False
1 no
2 windy == True
1 no
3 temp == mild
2 windy == False
1 no
4 humidity == normal
3 temp == mild
2 windy == True
1 yes
3 temp == cool
2 windy == False
1 yes
